In [ ]:
using Pkg

Pkg.activate()

inlcude("generate_ivt_fields.jl")

using preprocessing

In [ ]:
"""This function loads data in given geographic bounds. It supports loading values going 'over the end' like the lon rage from 270-40 NOTE: It is expected that the longitude is given in values from 0-360 deg and lat in range from -90:90""" 
function load_data_in_geo_bounds_typed(dataset, field_id::Union{String, Symbol, Missing}, geo_bnds::GeographicBounds, indices...; precision = Float32)::Array

  lon_normal_range = geo_bnds.lon_bounds[1] < geo_bnds.lon_bounds[2]
  lat_normal_range = geo_bnds.lat_bounds[1] < geo_bnds.lat_bounds[2]

  result_dim = length(indices) + 2


  if lon_normal_range & lat_normal_range
    return dataset[field_id][geo_bnds.lon_indices[1]:geo_bnds.lon_indices[2], geo_bnds.lat_indices[1]:geo_bnds.lat_indices[2], indices...]::Array{Union{precision, Missing}, result_dim}
  elseif !lon_normal_range & lat_normal_range
    
    lon_first = dataset[field_id][geo_bnds.lon_indices[1]:end, geo_bnds.lat_indices[1]:geo_bnds.lat_indices[2],indices...]::Array{Union{precision, Missing}, result_dim}
    lon_second = dataset[field_id][1:geo_bnds.lon_indices[2], geo_bnds.lat_indices[1]:geo_bnds.lat_indices[2],indices...]::Array{Union{precision, Missing}, result_dim}
    
    return vcat(lon_first, lon_second)::Array{Union{precision, Missing}, result_dim}
  elseif lon_normal_range & !lat_normal_range
    
    lat_first = dataset[field_id][geo_bnds.lon_indices[1]:geo_bnds.lon_indices[2], geo_bnds.lat_indices[1]:end,indices...]::Array{Union{precision, Missing}, result_dim}
    lat_second = dataset[field_id][geo_bnds.lon_indices[1]:geo_bnds.lon_indices[2], 1:geo_bnds.lat_indices[2],indices...]::Array{Union{precision, Missing}, result_dim}
    
    return hcat(lat_first, lat_second)::Array{Union{precision, Missing}, result_dim}
  else
    # last case is both are over 
    lon_f_lat_f = dataset[field_id][geo_bnds.lon_indices[1]:end, geo_bnds.lat_indices[1]:end,indices...]::Array{Union{precision, Missing}, result_dim}
    lon_f_lat_s = dataset[field_id][geo_bnds.lon_indices[1]:end, 1:geo_bnds.lat_indices[2],indices...]::Array{Union{precision, Missing}, result_dim}

    lon_s_lat_f = dataset[field_id][1:geo_bnds.lon_indices[2], geo_bnds.lat_indices[1]:end,indices...]::Array{Union{precision, Missing}, result_dim}
    lon_s_lat_s = dataset[field_id][1:geo_bnds.lon_indices[2], 1:geo_bnds.lat_indices[2],indices...]::Array{Union{precision, Missing}, result_dim}

    return vcat(hcat(lon_f_lat_f, lon_f_lat_s), hcat(lon_s_lat_f, lon_s_lat_s))::Array{Union{precision, Missing}, result_dim}
  end
end

In [1]:
using NCDatasets
using DataStructures

function create_benchmark_ds(path, varname, data)
    
    vertical_size = size(data, 3)

    ps = rand(size(data, 1), size(data, 2), size(data, 4))

    ap = rand(vertical_size)

    b = rand(vertical_size)

    NCDataset(path,"c",attrib = OrderedDict("title" => "this is a test file")) do ds
        # Define the variable temperature. The dimension "lon" and "lat" with the
        # size 100 and 110 resp are implicitly created
        defVar(ds,varname,data,("lon","lat", "lev", "time"))
        defVar(ds,"ps",ps,("lon","lat", "time"))
        defVar(ds,"ap",ap,("lev",))
        defVar(ds,"b",b,("lev",))
    end
    
end

create_benchmark_ds (generic function with 1 method)

In [2]:
data = [Float32(l+i+j/k) for i = 1:60, j = 1:50, k = 1:42, l = 1:5000]

println("Data size in mem: $(sizeof(data)/1000000)")
for id in ["hus", "ua", "va"]
    path = "sample_data/benchmark_$id.nc"
    create_benchmark_ds(path, id, data)
end



Data size in mem: 2520.0


In [1]:
using BenchmarkTools
using NCDatasets


bmable = @benchmarkable  NCDataset(["sample_data/benchmark_hus.nc", "sample_data/benchmark_va.nc", "sample_data/benchmark_ua.nc"]; aggdim = "") do ds

    data_hus = ds[:hus][:, :, :, :]
    data_ua = ds[:ua][:, :, :, :]
    data_va = ds[:va][:, :, :, :]
    data_ps = ds[:ps][:, :, :]

end

tune!(bmable)

run(bmable)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  4.199 s …    4.380 s  ┊ GC (min … max): 13.76% … 17.42%
 Time  (median):     4.289 s               ┊ GC (median):    15.63%
 Time  (mean ± σ):   4.289 s ± 128.221 ms  ┊ GC (mean ± σ):  15.63% ±  2.59%

  █                                                        █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  4.2 s          Histogram: frequency by time         4.38 s <

 Memory estimate: 7.15 GiB, allocs estimate: 3452.

In [2]:
bmable = @benchmarkable  NCDataset(["sample_data/benchmark_hus.nc", "sample_data/benchmark_va.nc", "sample_data/benchmark_ua.nc"]; aggdim = "") do ds

    data_hus = ds[:hus][1:40, 1:20, :, :]
    data_ua = ds[:ua][1:40, 1:20, :, :]
    data_va = ds[:va][1:40, 1:20, :, :]
    data_ps = ds[:ps][1:40, 1:20, :]

end samples = 15

tune!(bmable)

run(bmable)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.710 s …    1.932 s  ┊ GC (min … max):  6.43% … 17.18%
 Time  (median):     1.753 s               ┊ GC (median):     9.31%
 Time  (mean ± σ):   1.799 s ± 117.734 ms  ┊ GC (mean ± σ):  11.22% ±  5.56%

  █          █                                             █  
  █▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.71 s         Histogram: frequency by time         1.93 s <

 Memory estimate: 1.91 GiB, allocs estimate: 3452.

In [15]:
bmable = @benchmarkable  NCDataset(["sample_data/benchmark_hus.nc", "sample_data/benchmark_va.nc", "sample_data/benchmark_ua.nc"]; aggdim = "") do ds

    data_hus = ds[:hus][:, :, :, :]::Array{Float32, 4}
    # data_ua = ds[:ua][:, :, :, :]::Array{Float32, 4}
    # data_va = ds[:va][:, :, :, :]::Array{Float32, 4}
    # data_ps = ds[:ps][:, :, :]::Array{Float64, 3}

end

tune!(bmable)

run(bmable)

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.155 s …    1.416 s  ┊ GC (min … max): 0.09% … 18.81%
 Time  (median):     1.311 s               ┊ GC (median):    7.74%
 Time  (mean ± σ):   1.298 s ± 119.586 ms  ┊ GC (mean ± σ):  9.06% ±  9.76%

  █                   █                           █        █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█ ▁
  1.15 s         Histogram: frequency by time         1.42 s <

 Memory estimate: 2.35 GiB, allocs estimate: 2999.

In [20]:
bmable = @benchmarkable  NCDataset(["sample_data/benchmark_hus.nc", "sample_data/benchmark_va.nc", "sample_data/benchmark_ua.nc"]; aggdim = "") do ds

    data_hus = ds[:hus][1:40, 1:20, :, :]::Array{Float32, 4}
    data_ua = ds[:ua][1:40, 1:20, :, :]::Array{Float32, 4}
    data_va = ds[:va][1:40, 1:20, :, :]::Array{Float32, 4}
    data_ps = ds[:ps][1:40, 1:20, :]::Array{Float64, 3}

end

tune!(bmable)

run(bmable)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.843 s …    2.071 s  ┊ GC (min … max):  7.60% … 18.19%
 Time  (median):     1.899 s               ┊ GC (median):    10.83%
 Time  (mean ± σ):   1.937 s ± 118.653 ms  ┊ GC (mean ± σ):  12.43% ±  5.43%

  █             █                                          █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.84 s         Histogram: frequency by time         2.07 s <

 Memory estimate: 1.91 GiB, allocs estimate: 3452.

In [23]:
bmable = @benchmarkable  NCDataset(["sample_data/benchmark_hus.nc", "sample_data/benchmark_va.nc", "sample_data/benchmark_ua.nc"]; aggdim = "") do ds
# bmable = @benchmarkable  NCDataset("sample_data/benchmark_hus.nc") do ds


    data = Array{Float32, 5}

    

    for (i, id) in enumerate([:hus, :ua, :va])
        # NCDatasets.load!(ds[id], data[i, :, :, :, :], :, :, :, :)
        push!(data,ds[id][:, :, :, :])
    end
    data_ps = ds[:ps][:, :, :]::Array{Float64, 3}

end

tune!(bmable)

run(bmable)

MethodError: MethodError: no method matching push!(::Type{Array{Float32, 5}}, ::Array{Float32, 4})
Closest candidates are:
  push!(::Any, ::Any, !Matched::Any) at ~/.julia/juliaup/julia-1.7.0+0.x64.linux.gnu/share/julia/base/abstractarray.jl:2952
  push!(::Any, ::Any, !Matched::Any, !Matched::Any...) at ~/.julia/juliaup/julia-1.7.0+0.x64.linux.gnu/share/julia/base/abstractarray.jl:2953
  push!(!Matched::DataStructures.MutableLinkedList{T}, ::Any) where T at ~/.julia/packages/DataStructures/b0JVf/src/mutable_list.jl:199
  ...